### Stopping actors

#### 1. stop actor
stop actor 一般调用 `ActorContext` 或 `ActorSystem` 的 stop 方法:
* `context.stop` 用来停止自己或子actor; 
* `system.stop` 用来停止顶级 actor.   

真实的停止动作是异步的, 就是说 stop 方法可能会在 actor 被停止前返回

```scala
class MyActor extends Actor {
  val child: ActorRef = ???

  def receive = {
    case "interrupt-child" =>
      context.stop(child)

    case "done" =>
      context.stop(self)
  }
}
```
停止 actor 分为2步骤: 
1. 挂起 mailbox 操作, 发起 stop 命令到所有他的子 actor, 然后等待子 actor 的 termination 回馈, 直到所有子 actor 回馈完毕. 
2. 最后关闭自己
	* 调用 `postStop` 方法
	* 清空 mailbox
	* 发送 Terminated 信号给 `DeathWatch`
	* 告知自己的 supervisor
这个步骤确保 actor 系统子树有序的停止, stop 命令传播到叶子节点并等待其返回确认通知. 

#### 2. PoisonPill
可以通过向 actor 发送 `akka.actor.PoisonPill` 消息来停止一个 actor. `PoisonPill` 会想普通消息一样进入 mailbox 队列, 并且在 mailbox 中的已经存在的所有消息处理后执行. 因此, 与 stop 不同的是, `PoisonPill` 会等待mailbox消息处理完毕后再停止
```scala
p2 ! PoisonPill
```

#### 3. Killing an Actor
可以向 actor 发送 Kill 消息来杀死 actor, 这回导致 actor 抛出 `ActorKilledException` 异常, 因此可以被其 supervisor 检测到, 并决定是重启 actor 还是直接停掉actor (取决于监控策略) 
```scala
context.watch(victim) // watch the Actor to receive Terminated message once it dies
victim ! Kill
```

#### 4. 停止 actor 的推荐做法
不建议使用 `PoisonPill` 或 `Kill` 来设计 actor 的交互操作, 而是推荐使用类似 `PleaseCleanupAndStop` 这样的消息让 actor 知道如何处理停止. `PoisonPill` 或 `Kill` 是的无法控制 actor 的停止逻辑 /
